In [1]:
!pip install pandas openpyxl xlrd

Defaulting to user installation because normal site-packages is not writeable


In [36]:
import pandas as pd

# --- Step 1: Read Excel and skip top 7 rows ---
file_path = r"E:\OrderProcssing\data\AURA_ADAMS_PO.xls"
df = pd.read_excel(file_path, skiprows=7)

# --- Step 1a: Read cell B2 for ItemPoNo ---
item_po_no = pd.read_excel(file_path, header=None, nrows=2).iloc[1, 1]  # B2

# --- Step 2: Select specific columns ---
selected_columns = ['AURUM Code', 'REF SHIMAYRA', 'KT', 'SIZE', 'COLOUR', 'QUALITY', 'QTY']
df_selected = df[selected_columns]

# --- Step 3: Drop rows with NaN and make a copy ---
df_cleaned = df_selected.dropna().copy()

# --- Step 4: Rename columns ---
df_cleaned.rename(columns={
    'AURUM Code': 'SKUNo',
    'REF SHIMAYRA': 'StyleCode',
    'KT': 'Metal',
    'SIZE': 'ItemSize',
    'COLOUR': 'Tone',
    'QTY': 'OrderQty',
}, inplace=True)

# --- Step 5: Transform ItemSize values ---
def transform_itemsize(x):
    if pd.isna(x):
        return ''
    x = str(x).strip().upper()
    if x == 'NS':
        return ''
    if 'MM' in x:
        num_part = ''.join([ch for ch in x if ch.isdigit()])
        return f'UP{num_part}' if num_part else ''
    return x

df_cleaned['ItemSize'] = df_cleaned['ItemSize'].apply(transform_itemsize)

# --- Step 6: Metal mapping logic ---
def map_metal_tone(metal, tone):
    if pd.isna(metal) or pd.isna(tone):
        return ''
    metal = str(metal).upper()
    tone = str(tone).upper()

    if '18' in metal:
        if 'YELLOW' in tone:
            return 'G750Y'
        elif 'WHITE' in tone:
            return 'G750W'
        elif 'ROSE PINK' in tone or 'PINK' in tone:
            return 'G750P'

    if '14' in metal:
        if 'YELLOW' in tone:
            return 'G585Y'
        elif 'WHITE' in tone:
            return 'G585W'
        elif 'ROSE PINK' in tone or 'PINK' in tone:
            return 'G585P'

    if 'PLATINUM' in metal:
        return 'PC95'

    return metal

df_cleaned['Metal'] = df_cleaned.apply(lambda row: map_metal_tone(row['Metal'], row['Tone']), axis=1)

# --- Step 7: Update Tone column based on Metal postfix ---
def update_tone_from_metal(metal):
    if pd.isna(metal):
        return ''
    metal = str(metal).upper()
    if metal.endswith('Y'):
        return 'Y'
    elif metal.endswith('W'):
        return 'W'
    elif metal.endswith('P'):
        return 'P'
    elif 'PC95' in metal:
        return 'PT'
    return ''

df_cleaned['Tone'] = df_cleaned['Metal'].apply(update_tone_from_metal)

# --- Step 8: Add OrderItemPcs after OrderQty ---
df_cleaned['OrderItemPcs'] = ''

# --- Step 9: Add ItemPoNo after Tone ---
df_cleaned['ItemPoNo'] = item_po_no

# --- Step 10: Add new columns "ItemRefNo, StockType, Priority, MakeType" after ItemPoNo ---
priority_input = input("Enter Priority for all rows: ")
df_cleaned['ItemRefNo'] = ''
df_cleaned['StockType'] = ''
df_cleaned['Priority'] = priority_input
df_cleaned['MakeType'] = ''

# --- Step 11: Update MakeType based on Metal ---
def map_make_type(metal):
    mapping = {
        'G750W': '18K WHITE GOLD(OR JALINE 750)',
        'G750Y': '18K YELLOW GOLD(OR JALINE 750)',
        'G750P': '18K PINK GOLD(OR JALINE 750)',
        'G585W': '14K WHITE GOLD(OR JALINE 585)',
        'G585Y': '14K YELLOW GOLD(OR JALINE 585)',
        'G585P': '14K PINK GOLD(OR JALINE 585)',
        'PC95': 'PLATINUM(OR JALINE 95)'
    }
    return mapping.get(metal, '')

df_cleaned['CustomerProductionInstruction'] = df_cleaned['Metal'].apply(map_make_type)


# --- Step 13: Add SpecialRemarks after CustomerProductionInstruction ---
def create_special_remarks(row):
    sku = row['SKUNo']
    metal_desc_map = {
        'G750W': '750 WHITE GOLD',
        'G750Y': '750 YELLOW GOLD',
        'G750P': '750 PINK GOLD',
        'G585W': '585 WHITE GOLD',
        'G585Y': '585 YELLOW GOLD',
        'G585P': '585 PINK GOLD',
        'PC95': '95 PLATINUM'
    }
    metal_desc = metal_desc_map.get(row['Metal'], '')

    size_desc = ''
    if str(row['ItemSize']).upper().startswith('UP'):
        num = ''.join([c for c in row['ItemSize'] if c.isdigit()])
        if num:
            size_desc = f'SZ-{num}'

    quality = row['QUALITY'] if pd.notna(row['QUALITY']) else ''
    parts = [sku]
    if metal_desc:
        parts.append(metal_desc)
    if size_desc:
        parts.append(size_desc)
    if quality:
        parts.append(f'DIA QLTY: {quality}')
    return ', '.join(parts)

df_cleaned['SpecialRemarks'] = df_cleaned.apply(create_special_remarks, axis=1)

# --- Step 14: Add DesignProductionInstruction after SpecialRemarks ---
def map_design_instruction(tone):
    if pd.isna(tone):
        return ''
    tone = str(tone).upper()
    if tone == 'W':
        return 'WHITE RODIUM'
    elif tone in ['Y', 'P', 'PT']:
        return 'NO RODIUM'
    return ''

df_cleaned['DesignProductionInstruction'] = df_cleaned['Tone'].apply(map_design_instruction)

def create_stamp_instruction(row):
    special = str(row['SpecialRemarks']).upper()
    if '750' in special:
        metal_number = '750'
    elif '585' in special:
        metal_number = '585'
    elif '95' in special:
        metal_number = '95'
    else:
        metal_number = ''
    
    if metal_number:
        return f'CUSTOMER LOGO & {metal_number}'
    return ''

df_cleaned['StampInstruction'] = df_cleaned.apply(create_stamp_instruction, axis=1)

# --- Step 16: Add OrderGroup and Certificate after StampInstruction ---
df_cleaned['OrderGroup'] = ''
df_cleaned['Certificate'] = ''

# --- Step 19: Add new columns after SKUNo ---
new_cols_after_sku = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 
    'Basemetalmaxwt', 'Productiondeliverydate', 'Expecteddeliverydate'
]

for col in new_cols_after_sku:
    df_cleaned[col] = ''

# --- Step 20: Reorder columns with the new ones after SKUNo ---
column_order = [
    'StyleCode', 'ItemSize', 'OrderQty', 'OrderItemPcs',
    'Metal', 'Tone', 'ItemPoNo', 'ItemRefNo', 'StockType', 'Priority',
    'MakeType', 'CustomerProductionInstruction', 'SpecialRemarks', 
    'DesignProductionInstruction', 'StampInstruction', 'OrderGroup', 'Certificate',
    'SKUNo', 
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 
    'Basemetalmaxwt', 'Productiondeliverydate', 'Expecteddeliverydate',
    'QUALITY'
]

df_cleaned = df_cleaned[column_order]
df_cleaned.drop(columns=['QUALITY'], inplace=True) 

# --- Step 17: Display first row ---
df_cleaned.head(1) 

df_cleaned.to_csv(r"C:\Users\Admin\Desktop\AAM_PO_Cleaned.csv", index=False)

Enter Priority for all rows:  REG
